In [100]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jun 15 14:47:41 2018

@author: niccolop
"""
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk 

#%% load data 
"""
This data was collected on 15th June 2018. They represent the first 10 websites 
suggested by Google from the query "pro life arguments" and "pro choice 
arguments", plus an 11th entry for the each argument as summarized by Wikipedia.
Websites were filtered based on their content: e.g. some pro life arguments 
came after querying for pro choice. Or e.g. some websites had both arguments 
summarized."""
texts=[]
for w in range(10):
    texts.append(open('./data_files/abortion/against/web' + str(w+1) + '.txt', 'r').read())
for w in range(10):
    texts.append(open('./data_files/abortion/pro/web' + str(w+1) + '.txt', 'r').read())
    
#%% clean doc
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
import re
from pprint import pprint  # pretty-printer

# remove urls
texts = [re.sub(r"http\S+", "", doc) for doc in texts]
texts = [' '.join(item for item in doc.split() if not (item.startswith('www.')))  for doc in texts]
# strip trailing symbols
texts = [doc.replace('\n','') for doc in texts]
# remove double quotes
texts = [doc.replace('—',' ').replace('–',' ') for doc in texts]
# remove numbers
texts = [re.sub(r'\d+', '', doc) for doc in texts]
    
# remove punctuation and stopwords
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) | set('“”"’')
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

def clean_sent(doc):
    stop_free = ([i for i in doc.lower().split('.')])
    return stop_free

# tokenize docs
#texts_clean = [clean_sent(doc) for doc in texts]  

texts_clean = [clean_sent(doc) for doc in texts]  

# remove non english words
words = set(nltk.corpus.words.words())
for i,doc in enumerate(texts_clean):
    texts_clean[i] = [w for w in doc if w in words or not w.isalpha()]

# this is useless...
texts_corpus = list(itertools.chain.from_iterable(texts_clean))

#%% 
# Importing Gensim
from gensim import corpora, models, similarities

#Creating the term dictionary of our courpus, where every unique term 
#is assigned an index. 
dictionary_against = corpora.Dictionary(texts_clean[:10]) 
dictionary_pro = corpora.Dictionary(texts_clean[10:]) 
dictionary = corpora.Dictionary(texts_clean) 
# Converting list of documents (corpus) into Document Term Matrix using 
# dictionary prepared above.
corpus_against = [dictionary.doc2bow(doc) for doc in texts_clean[:10]]
corpus_pro = [dictionary.doc2bow(doc) for doc in texts_clean[10:]]

In [107]:
pro = [item for sublist in texts_clean[0:10] for item in sublist]
against = [item for sublist in texts_clean[10:] for item in sublist]

In [109]:
import keras
tokenizer = keras.preprocessing.text.Tokenizer(num_words=2000, lower=True,split=" ")
tokenizer.fit_on_texts(pro + against)
p_encoded = tokenizer.texts_to_sequences(pro)
a_encoded = tokenizer.texts_to_sequences(against)

In [108]:
pro[1]

' millions of american women have aborted a child, and the pain, loss, and emotional need to justify what was done, both on the part of the mother and on the part of her loved ones, is strong and deep'

In [77]:
against[100]

'  peer-reviewed study comparing the mental health of women who received abortions to women denied abortions found that women who were denied abortions "felt more regret and anger" and "less relief and happiness" than women who had abortions'

In [113]:
print(p_encoded[1])

[1063, 2, 154, 27, 21, 341, 4, 46, 5, 1, 182, 5, 579, 267, 3, 1794, 49, 47, 378, 162, 17, 1, 225, 2, 1, 117, 5, 17, 1, 225, 2, 44, 742, 8, 743, 5, 744]


In [161]:
p

[9, 8, 1356, 78, 83, 127, 1357, 340]

In [159]:
csr_matrix((0*p+1, (0*p, p)), shape=(1, 2000)).toarray()

TypeError: can only concatenate list (not "int") to list

In [170]:
from scipy.sparse import csr_matrix
p_enc = np.array([csr_matrix((0*np.array(p)+1, (0*np.array(p), np.array(p))), shape=(1, 2000)).toarray() for p in p_encoded])
a_enc = np.array([csr_matrix((0*np.array(p)+1, (0*np.array(p), np.array(p))), shape=(1, 2000)).toarray() for p in a_encoded])

In [206]:
p_enc = p_enc.squeeze()
a_enc = a_enc.squeeze()

print(p_enc[1])
print(p_enc[1].shape)
X = np.vstack((p_enc,a_enc))
y = np.hstack((p_enc[:,0]*0,a_enc[:,0]*0 + 1))

[ 0.  4.  3. ...,  0.  0.  0.]
(2000,)


In [208]:
print(y.shape), print(np.mean(y))

(1855,)
0.311051212938


(None, None)

In [217]:
lr = linear_model.LogisticRegression()
lr.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [219]:
from sklearn.metrics import confusion_matrix
y_true = y
y_pred = lr.predict(X)
print(confusion_matrix(y_true, y_pred))

0.0
[[1274    4]
 [  92  485]]


In [185]:
## LR
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.datasets import make_classification

# Create the RFE object and compute a cross-validated score.
lr = linear_model.Lasso(alpha = 0.1)
# The "accuracy" scoring is proportional to the number of correct
# classifications
rfecv = RFECV(estimator=lr, step=2, cv=StratifiedKFold(2),
              scoring='accuracy')
rfecv.fit(X[train], y[train])

print("Optimal number of features : %d" % rfecv.n_features_)

# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import confusion_matrix
y_true = y[test]
y_pred = RFECV.predict(X[test])
confusion_matrix(y_true, y_pred)

In [92]:
pro = [item for sublist in corpus_pro for item in sublist]
against = [item for sublist in corpus_against for item in sublist]

In [103]:
len(corpus_pro)

10

In [ ]:
## LR
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.datasets import make_classification

# Build a classification task using 3 informative features
X, y = make_classification(n_samples=1000, n_features=25, n_informative=3,
                           n_redundant=2, n_repeated=0, n_classes=8,
                           n_clusters_per_class=1, random_state=0)

# Create the RFE object and compute a cross-validated score.
lr = linear_model.Lasso(alpha = 0.1)
# The "accuracy" scoring is proportional to the number of correct
# classifications
rfecv = RFECV(estimator=lr, step=2, cv=StratifiedKFold(2),
              scoring='accuracy')
rfecv.fit(X, y)

print("Optimal number of features : %d" % rfecv.n_features_)

# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

In [ ]:

# remove non english words
words = set(nltk.corpus.words.words())
for i,doc in enumerate(texts_clean):
    texts_clean[i] = [w for w in doc if w in words or not w.isalpha()]

# this is useless...
texts_corpus = list(itertools.chain.from_iterable(texts_clean))

#%% 
# Importing Gensim
from gensim import corpora, models, similarities

#Creating the term dictionary of our courpus, where every unique term 
#is assigned an index. 
dictionary_against = corpora.Dictionary(texts_clean[0:10]) 
dictionary_pro = corpora.Dictionary(texts_clean[11:-1]) 
dictionary = corpora.Dictionary(texts_clean) 
# Converting list of documents (corpus) into Document Term Matrix using 
# dictionary prepared above.
corpus_against = [dictionary.doc2bow(doc) for doc in texts_clean[0:10]]
corpus_pro = [dictionary.doc2bow(doc) for doc in texts_clean[11:-1]]


In [10]:
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.datasets import make_classification

In [41]:
pro = np.load('pro.npy')
against = np.load('against.npy')
pro = [item for sublist in pro for item in sublist]
against = [item for sublist in against for item in sublist]
pro = [p  for p in pro if p is not None]
against = [p  for p in against if p is not None]

In [51]:
import keras
tokenizer = keras.preprocessing.text.Tokenizer(num_words=2000, lower=True,split=" ")
tokenizer.fit_on_texts(pro + against)
p_encoded = tokenizer.texts_to_sequences(pro)
a_encoded = tokenizer.texts_to_sequences(against)

In [54]:
print(a_encoded[1])

[2, 158, 23, 24, 392, 4, 45, 5, 1, 191, 5, 624, 279, 3, 47, 49, 425, 162, 18, 1, 236, 2, 1, 105, 5, 18, 1, 236, 2, 46, 1763, 8, 1675, 5, 1747]


In [56]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit(p_encoded)

p = enc.transform(p_encoded)



ValueError: setting an array element with a sequence.

In [12]:
X.shape

(1000, 25)

In [ ]:
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.datasets import make_classification

# Build a classification task using 3 informative features
X, y = make_classification(n_samples=1000, n_features=25, n_informative=3,
                           n_redundant=2, n_repeated=0, n_classes=8,
                           n_clusters_per_class=1, random_state=0)

# Create the RFE object and compute a cross-validated score.
lr = linear_model.Lasso(alpha = 0.1)
# The "accuracy" scoring is proportional to the number of correct
# classifications
rfecv = RFECV(estimator=lr, step=2, cv=StratifiedKFold(2),
              scoring='accuracy')
rfecv.fit(X, y)

print("Optimal number of features : %d" % rfecv.n_features_)

# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()